In [1]:
# !pip list | grep datasets
# !pip list | grep sentencepiece
# !pip list | grep transformers
# !pip list | grep tqdm

datasets                     2.12.0
sentencepiece                0.1.99
transformers                 4.29.0
tqdm                         4.65.0


In [2]:
# !pip install bitsandbytes
# !pip install loralib
# !pip install -q git+https://github.com/huggingface/peft.git
# !pip install --upgrade accelerate
# !pip install -U tqdm

In [1]:
import os

import torch
from torch import nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers as T
from transformers import AutoTokenizer, AutoConfig
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model, get_peft_model_state_dict

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/opt/aws-ofi-nccl/lib:/usr/local/lib:/usr/lib: did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('pmix-server.2040307;tcp4'), PosixPath('//127.0.0.1'), PosixPath('56529')}
  warn(msg)
/fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/share/modules/$MODULE_VERSION/modulefiles'), PosixPath('1')}
  warn(msg)
/fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:

In [2]:
MICRO_BATCH_SIZE = 4
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 3 
LEARNING_RATE = 3e-4
CUTOFF_LEN = 256
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
VAL_SET_SIZE = 2000

In [5]:
model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [6]:
tokenizer = LlamaTokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf", add_eos_token=True,
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [7]:
model = prepare_model_for_int8_training(model)

In [8]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
tokenizer.pad_token_id = 0

In [9]:
data = load_dataset("json", data_files="japanese_alpaca_data.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-88fb7e0d1f58050f/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
train_val = data["train"].train_test_split(
    test_size=VAL_SET_SIZE, shuffle=True, seed=42
)
train_data = train_val["train"]
val_data = train_val["test"]

In [12]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Response:
{data_point["output"]}"""
    
def tokenize(prompt):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"][:-1],
        "attention_mask": result["attention_mask"][:-1],
    }

In [13]:
train_data = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x)))
val_data = val_data.shuffle().map(lambda x: tokenize(generate_prompt(x)))

Map:   0%|          | 0/50002 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [14]:
world_size = int(os.environ.get('WORLD_SIZE', 1))
ddp = world_size != 1

In [15]:
ddp

False

In [16]:
trainer = T.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=T.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=20,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=200,
        save_steps=200,
        output_dir="../outputs/japanese-lora-alpaca_train_1",
        save_total_limit=3,
        load_best_model_at_end=True,
        ddp_find_unused_parameters=False if ddp else None,
    ),
    data_collator=T.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.8/site-packages/transformers/utils/import_utils.py:1146 in _get_module    │
│                                                                                                  │
│   1143 │                                                                                         │
│   1144 │   def _get_module(self, module_name: str):                                              │
│   1145 │   │   try:                                                                              │
│ ❱ 1146 │   │   │   return importlib.import_module("." + module_name, self.__name__)              │
│   1147 │   │   except Exception as e:                                                            │
│   1148 │   │   │   raise RuntimeError(                                                           │
│   1149 │   │   │   │   f"Failed to import {self.__name__}.{module_name} because of the followin  │
│                                                                                                  │
│ /opt/conda/lib/python3.8/importlib/__init__.py:127 in import_module                              │
│                                                                                                  │
│   124 │   │   │   if character != '.':                                                           │
│   125 │   │   │   │   break                                                                      │
│   126 │   │   │   level += 1                                                                     │
│ ❱ 127 │   return _bootstrap._gcd_import(name[level:], package, level)                            │
│   128                                                                                            │
│   129                                                                                            │
│   130 _RELOADING = {}                                                                            │
│ in _gcd_import                                                                                   │
│ in _find_and_load                                                                                │
│ in _find_and_load_unlocked                                                                       │
│ in _load_unlocked                                                                                │
│ in exec_module                                                                                   │
│ in _call_with_frames_removed                                                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.8/site-packages/transformers/trainer.py:173 in <module>                   │
│                                                                                                  │
│    170 │   DEFAULT_PROGRESS_CALLBACK = NotebookProgressCallback                                  │
│    171                                                                                           │
│    172 if is_apex_available():                                                                   │
│ ❱  173 │   from apex import amp                                                                  │
│    174                                                                                           │
│    175 if is_datasets_available():                                                               │
│    176 │   import datasets                                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import name 'amp' from 'apex' (unknown location)

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>   

In [20]:
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

/home/fnakamura/workspace/apex
Using pip 23.1.2 from /opt/conda/lib/python3.8/site-packages/pip (python 3.8)
DEPRECATION: --build-option and --global-option are deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11859
Processing /home/fnakamura/workspace/apex
  Running command python setup.py egg_info


  torch.__version__  = 1.13.1+cu117


  running egg_info
  creating /tmp/pip-pip-egg-info-h85js14d/apex.egg-info
  writing /tmp/pip-pip-egg-info-h85js14d/apex.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-h85js14d/apex.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-h85js14d/apex.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-h85js14d/apex.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-h85js14d/apex.egg-info/SOURCES.txt'
  writing manifest file '/tmp/pip-pip-egg-info-

## Evaluation

In [3]:
torch.cuda.is_available()

True

In [4]:
!ls /fsx/jp-llm/instruction_tuning/outputs/japanese-lora-alpaca_train_1/japanese-lora-alpaca

adapter_config.json  adapter_model.bin


In [5]:
from peft import PeftModel
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig

In [6]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model_custom = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model_original = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model_custom = PeftModel.from_pretrained(model_custom, "/fsx/jp-llm/instruction_tuning/outputs/japanese-lora-alpaca_train_1/japanese-lora-alpaca")
model_original = PeftModel.from_pretrained(model_original, "tloen/alpaca-lora-7b")
     

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [1]:
def generate_instruction_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [8]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        # print("Output: ", output)
        # print("-----")
        response = output.split("### Response:")[1].strip()
        if "Instruction" in response:
            response = response.split("### Instruction")[0].strip()
        print("Response:", response)
        # output.split("### Response:")[1].strip().split("### Instruction")[0].strip()

In [9]:
evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
日本の首都はどこですか？

### Response:
-----
Response: 日本の首都は東京です。


In [10]:
evaluate(model_custom,"アメリカの首都はどこですか？")

Prompt:  Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
アメリカの首都はどこですか？

### Response:
-----
Response: アメリカの首都はニューヨークです。


In [11]:
evaluate(model_custom, "ロシアとウクライナの戦争はいつ終わりますか？")

Prompt:  Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
ロシアとウクライナの戦争はいつ終わりますか？

### Response:
-----
Response: ロシアとウクライナの戦争はいつ終わりますか？
ロシアとウクライナの戦争はいつ終わりますか？
ロシアとウクライナの戦争はいつ終わりますか？
ロシアとウクライナの戦争はいつ終わりますか？
ロシアとウクライナの戦争はいつ終わりますか？
ロシアとウクライナの戦争はいつ終わりますか？
ロシアとウクライナの戦争はいつ終わりますか？
ロシアとウクライナの戦争はいつ終わりますか？
ロシアとウクライナの戦争はいつ終わりますか？
ロシアとウクライナの戦�


In [12]:
evaluate(model_custom, "伊藤博文とは誰ですか？")

Prompt:  Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
伊藤博文とは誰ですか？

### Response:
-----
Response: 伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博文とは誰ですか？
伊藤博


In [13]:
evaluate(model_custom,"アメリカの首都はどこですか？")

Prompt:  Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
アメリカの首都はどこですか？

### Response:
-----
Response: アメリカの首都はニューヨークです。


In [43]:
!ls /fsx/jp-llm/hf_model/test/

config.json  generation_config.json  pytorch_model.bin


In [49]:
!ls /fsx/jp-llm/

ckpts	  hf_model  novelai-spm-merges.txt  novelai.model
datasets  merged    novelai-spm-vocab.json  polyglot-ja


In [46]:
import sentencepiece as spm
from transformers import GPTNeoXForCausalLM, GPTNeoXConfig
import torch

In [45]:
def load_model(model_dir,):
    config = GPTNeoXConfig.from_pretrained(model_dir)
    config.is_decoder = True
    model = GPTNeoXForCausalLM.from_pretrained(model_dir, config=config, torch_dtype=torch.bfloat16)
    if torch.cuda.is_available():
        model = model.to("cuda:0")
    return model

In [47]:
model = load_model("/fsx/jp-llm/hf_model/test/")

In [48]:
type(model)

transformers.models.gpt_neox.modeling_gpt_neox.GPTNeoXForCausalLM

In [50]:
spm = spm.SentencePieceProcessor(model_file="/fsx/jp-llm/novelai.model")

In [112]:
spm.pad_id()

0

In [51]:
type(spm)

sentencepiece.SentencePieceProcessor

In [113]:
?spm.encode

In [60]:
def generate_instruction_prompt(instruction, _input=None):
    if _input:
        return f"""以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 命令:
{instruction}

### 入力:
{_input}

### 応答:"""
    else:
        return f"""以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
{instruction}

### 応答:"""

In [91]:
instruction = "アメリカの首都はどこですか？"
prompt = generate_instruction_prompt(instruction, _input=None)
print(prompt)

以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
アメリカの首都はどこですか？

### 応答:


In [92]:
model.device

device(type='cuda', index=0)

In [93]:
tokens = spm.encode(prompt)
input_ids = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(model.device)

In [94]:
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=384,
)

In [96]:
outputs = []
for s in generation_output.sequences[0]:
    output = spm.decode_ids(s.item())
    outputs.append(output)
#     response = output.split("### Response:")[1].strip()
#     if "Instruction" in response:
#         response = response.split("### Instruction")[0].strip()
#     print("Response:", response)

In [90]:
print(prompt)

以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
アメリカの首都はどこですか？

### 応答:


In [99]:
print("".join(outputs).replace(prompt, ""))


アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？
### 応答:
アメリカの首都はどこですか？


In [100]:
instruction = "日本の首都はどこですか？"
prompt = generate_instruction_prompt(instruction, _input=None)
print(prompt)

以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:


In [101]:
tokens = spm.encode(prompt)
input_ids = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(model.device)

In [102]:
generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=384,
    )

In [103]:
outputs = []
for s in generation_output.sequences[0]:
    output = spm.decode_ids(s.item())
    outputs.append(output)
#     response = output.split("### Response:")[1].strip()
#     if "Instruction" in response:
#         response = response.split("### Instruction")[0].strip()
#     print("Response:", response)

In [104]:
print("".join(outputs).replace(prompt, ""))


日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？
### 応答:
日本の都市はどこですか？


In [105]:
prompt = "日本の首都は、"
tokens = spm.encode(prompt)
input_ids = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(model.device)

In [106]:
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=128,
)

In [107]:
outputs = []
for s in generation_output.sequences[0]:
    output = spm.decode_ids(s.item())
    outputs.append(output)
#     response = output.split("### Response:")[1].strip()
#     if "Instruction" in response:
#         response = response.split("### Instruction")[0].strip()
#     print("Response:", response)

In [108]:
print("".join(outputs).replace(prompt, ""))

首都機能が集中する東京です。首都機能が集中する東京は、日本の政治、経済、文化の中心地であり、日本の政治、経済、文化の中心地です。東京は、日本の政治、経済、文化の中心地であり、日本の政治、経済、文化の中心地です。東京は、日本の政治、経済、文化の中心地であり、日本の政治、経済、文化の中心地です。東京は、日本の政治、経済、文化の中心地であり、日本の政治、経済、文化の中心地です。東京は、日本の政治、経済、文化の中心地であり、日本の政治、経済、文化の中心地です。


In [114]:
from transformers import GPTNeoXTokenizerFast

In [116]:
tokenizer = GPTNeoXTokenizerFast.from_pretrained("/fsx/jp-llm/nai_tokenizer")

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_2042734/3619849896.py:1 in <module>                                        │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2042734/3619849896.py'               │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/transformers/tokenization_uti │
│ ls_base.py:1796 in from_pretrained                                                        │
│                                                                                           │
│   1793 │   │   │   )                                                                      │
│   1794 │   │                                                                              │
│   1795 │   │   if all(full_file_name is None for full_file_name in resolved_vocab_files.v │
│ ❱ 1796 │   │   │   raise EnvironmentError(                                                │
│   1797 │   │   │   │   f"Can't load tokenizer for '{pretrained_model_name_or_path}'. If y │
│   1798 │   │   │   │   "'https://huggingface.co/models', make sure you don't have a local │
│   1799 │   │   │   │   f"Otherwise, make sure '{pretrained_model_name_or_path}' is the co │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
OSError: Can't load tokenizer for '/fsx/jp-llm/nai_tokenizer'. If you were trying to load it 
from 'https://huggingface.co/models', make sure you don't have a local directory with the 
same name. Otherwise, make sure '/fsx/jp-llm/nai_tokenizer' is the correct path to a 
directory containing all relevant files for a GPTNeoXTokenizerFast tokenizer.

In [118]:
tokenizer = GPTNeoXTokenizerFast(
    vocab_file="/fsx/jp-llm/nai_tokenizer/novelai-spm-vocab.json",
    merges_file="/fsx/jp-llm/nai_tokenizer/novelai-spm-merges.txt",
    tokenizer_file="/fsx/jp-llm/nai_tokenizer/novelai.model",
)

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_2042734/2456364245.py:1 in <module>                                        │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2042734/2456364245.py'               │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/transformers/models/gpt_neox/ │
│ tokenization_gpt_neox_fast.py:113 in __init__                                             │
│                                                                                           │
│   110 │   │   add_prefix_space=False,                                                     │
│   111 │   │   **kwargs,                                                                   │
│   112 │   ):                                                                              │
│ ❱ 113 │   │   super().__init__(                                                           │
│   114 │   │   │   vocab_file,                                                             │
│   115 │   │   │   merges_file,                                                            │
│   116 │   │   │   tokenizer_file=tokenizer_file,                                          │
│                                                                                           │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/transformers/tokenization_uti │
│ ls_fast.py:111 in __init__                                                                │
│                                                                                           │
│   108 │   │   │   fast_tokenizer = copy.deepcopy(tokenizer_object)                        │
│   109 │   │   elif fast_tokenizer_file is not None and not from_slow:                     │
│   110 │   │   │   # We have a serialization from tokenizers which let us directly build t │
│ ❱ 111 │   │   │   fast_tokenizer = TokenizerFast.from_file(fast_tokenizer_file)           │
│   112 │   │   elif slow_tokenizer is not None:                                            │
│   113 │   │   │   # We need to convert a slow tokenizer to build the backend              │
│   114 │   │   │   fast_tokenizer = convert_slow_tokenizer(slow_tokenizer)                 │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
Exception: stream did not contain valid UTF-8

In [119]:
tokenizer = GPTNeoXTokenizerFast(
    vocab_file="/fsx/jp-llm/nai_tokenizer/novelai-spm-vocab.json",
    merges_file="/fsx/jp-llm/nai_tokenizer/novelai-spm-merges.txt",
    # tokenizer_file="/fsx/jp-llm/nai_tokenizer/novelai.model",
)

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_2042734/243260755.py:1 in <module>                                         │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2042734/243260755.py'                │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/transformers/models/gpt_neox/ │
│ tokenization_gpt_neox_fast.py:113 in __init__                                             │
│                                                                                           │
│   110 │   │   add_prefix_space=False,                                                     │
│   111 │   │   **kwargs,                                                                   │
│   112 │   ):                                                                              │
│ ❱ 113 │   │   super().__init__(                                                           │
│   114 │   │   │   vocab_file,                                                             │
│   115 │   │   │   merges_file,                                                            │
│   116 │   │   │   tokenizer_file=tokenizer_file,                                          │
│                                                                                           │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/transformers/tokenization_uti │
│ ls_fast.py:120 in __init__                                                                │
│                                                                                           │
│   117 │   │   │   slow_tokenizer = self.slow_tokenizer_class(*args, **kwargs)             │
│   118 │   │   │   fast_tokenizer = convert_slow_tokenizer(slow_tokenizer)                 │
│   119 │   │   else:                                                                       │
│ ❱ 120 │   │   │   raise ValueError(                                                       │
│   121 │   │   │   │   "Couldn't instantiate the backend tokenizer from one of: \n"        │
│   122 │   │   │   │   "(1) a `tokenizers` library serialization file, \n"                 │
│   123 │   │   │   │   "(2) a slow tokenizer instance to convert or \n"                    │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [129]:
!pip install -U sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.1 MB/s eta 0:00:000:00:01
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.97
    Uninstalling sentencepiece-0.1.97:
      Successfully uninstalled sentencepiece-0.1.97


In [130]:
tokenizer = GPTNeoXTokenizerFast(
    vocab_file="/fsx/jp-llm/nai_tokenizer/novelai-spm-vocab.json",
    merges_file="/fsx/jp-llm/nai_tokenizer/novelai-spm-merges.txt",
    # tokenizer_file="/fsx/jp-llm/nai_tokenizer/novelai.model",
)

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_2042734/243260755.py:1 in <module>                                         │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2042734/243260755.py'                │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/transformers/models/gpt_neox/ │
│ tokenization_gpt_neox_fast.py:113 in __init__                                             │
│                                                                                           │
│   110 │   │   add_prefix_space=False,                                                     │
│   111 │   │   **kwargs,                                                                   │
│   112 │   ):                                                                              │
│ ❱ 113 │   │   super().__init__(                                                           │
│   114 │   │   │   vocab_file,                                                             │
│   115 │   │   │   merges_file,                                                            │
│   116 │   │   │   tokenizer_file=tokenizer_file,                                          │
│                                                                                           │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/transformers/tokenization_uti │
│ ls_fast.py:120 in __init__                                                                │
│                                                                                           │
│   117 │   │   │   slow_tokenizer = self.slow_tokenizer_class(*args, **kwargs)             │
│   118 │   │   │   fast_tokenizer = convert_slow_tokenizer(slow_tokenizer)                 │
│   119 │   │   else:                                                                       │
│ ❱ 120 │   │   │   raise ValueError(                                                       │
│   121 │   │   │   │   "Couldn't instantiate the backend tokenizer from one of: \n"        │
│   122 │   │   │   │   "(1) a `tokenizers` library serialization file, \n"                 │
│   123 │   │   │   │   "(2) a slow tokenizer instance to convert or \n"                    │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [123]:
tokenizer = GPTNeoXTokenizerFast(
    vocab_file="/fsx/jp-llm/nai_tokenizer/novelai-spm-vocab.json",
    merges_file="/fsx/jp-llm/nai_tokenizer/novelai-spm-merges.txt",
    tokenizer_file="/fsx/jp-llm/nai_tokenizer/novelai.model",
)

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_2042734/2456364245.py:1 in <module>                                        │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2042734/2456364245.py'               │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/transformers/models/gpt_neox/ │
│ tokenization_gpt_neox_fast.py:113 in __init__                                             │
│                                                                                           │
│   110 │   │   add_prefix_space=False,                                                     │
│   111 │   │   **kwargs,                                                                   │
│   112 │   ):                                                                              │
│ ❱ 113 │   │   super().__init__(                                                           │
│   114 │   │   │   vocab_file,                                                             │
│   115 │   │   │   merges_file,                                                            │
│   116 │   │   │   tokenizer_file=tokenizer_file,                                          │
│                                                                                           │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/transformers/tokenization_uti │
│ ls_fast.py:111 in __init__                                                                │
│                                                                                           │
│   108 │   │   │   fast_tokenizer = copy.deepcopy(tokenizer_object)                        │
│   109 │   │   elif fast_tokenizer_file is not None and not from_slow:                     │
│   110 │   │   │   # We have a serialization from tokenizers which let us directly build t │
│ ❱ 111 │   │   │   fast_tokenizer = TokenizerFast.from_file(fast_tokenizer_file)           │
│   112 │   │   elif slow_tokenizer is not None:                                            │
│   113 │   │   │   # We need to convert a slow tokenizer to build the backend              │
│   114 │   │   │   fast_tokenizer = convert_slow_tokenizer(slow_tokenizer)                 │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
Exception: stream did not contain valid UTF-8

In [126]:
T.AutoTokenizer.from_pretrained("/fsx/jp-llm/stablelm-jp")

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_2042734/2009909866.py:1 in <module>                                        │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2042734/2009909866.py'               │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/transformers/models/auto/toke │
│ nization_auto.py:711 in from_pretrained                                                   │
│                                                                                           │
│   708 │   │   if model_type is not None:                                                  │
│   709 │   │   │   tokenizer_class_py, tokenizer_class_fast = TOKENIZER_MAPPING[type(confi │
│   710 │   │   │   if tokenizer_class_fast and (use_fast or tokenizer_class_py is None):   │
│ ❱ 711 │   │   │   │   return tokenizer_class_fast.from_pretrained(pretrained_model_name_o │
│   712 │   │   │   else:                                                                   │
│   713 │   │   │   │   if tokenizer_class_py is not None:                                  │
│   714 │   │   │   │   │   return tokenizer_class_py.from_pretrained(pretrained_model_name │
│                                                                                           │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/transformers/tokenization_uti │
│ ls_base.py:1796 in from_pretrained                                                        │
│                                                                                           │
│   1793 │   │   │   )                                                                      │
│   1794 │   │                                                                              │
│   1795 │   │   if all(full_file_name is None for full_file_name in resolved_vocab_files.v │
│ ❱ 1796 │   │   │   raise EnvironmentError(                                                │
│   1797 │   │   │   │   f"Can't load tokenizer for '{pretrained_model_name_or_path}'. If y │
│   1798 │   │   │   │   "'https://huggingface.co/models', make sure you don't have a local │
│   1799 │   │   │   │   f"Otherwise, make sure '{pretrained_model_name_or_path}' is the co │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
OSError: Can't load tokenizer for '/fsx/jp-llm/stablelm-jp'. If you were trying to load it 
from 'https://huggingface.co/models', make sure you don't have a local directory with the 
same name. Otherwise, make sure '/fsx/jp-llm/stablelm-jp' is the correct path to a directory 
containing all relevant files for a GPTNeoXTokenizerFast tokenizer.

In [128]:
tokenizer = GPTNeoXTokenizerFast(
#     vocab_file="/fsx/jp-llm/nai_tokenizer/novelai-spm-vocab.json",
#     merges_file="/fsx/jp-llm/nai_tokenizer/novelai-spm-merges.txt",
    tokenizer_file="/fsx/jp-llm/stablelm-jp/novelai.model",
)

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_2042734/2427677215.py:1 in <module>                                        │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2042734/2427677215.py'               │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/transformers/models/gpt_neox/ │
│ tokenization_gpt_neox_fast.py:113 in __init__                                             │
│                                                                                           │
│   110 │   │   add_prefix_space=False,                                                     │
│   111 │   │   **kwargs,                                                                   │
│   112 │   ):                                                                              │
│ ❱ 113 │   │   super().__init__(                                                           │
│   114 │   │   │   vocab_file,                                                             │
│   115 │   │   │   merges_file,                                                            │
│   116 │   │   │   tokenizer_file=tokenizer_file,                                          │
│                                                                                           │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/transformers/tokenization_uti │
│ ls_fast.py:111 in __init__                                                                │
│                                                                                           │
│   108 │   │   │   fast_tokenizer = copy.deepcopy(tokenizer_object)                        │
│   109 │   │   elif fast_tokenizer_file is not None and not from_slow:                     │
│   110 │   │   │   # We have a serialization from tokenizers which let us directly build t │
│ ❱ 111 │   │   │   fast_tokenizer = TokenizerFast.from_file(fast_tokenizer_file)           │
│   112 │   │   elif slow_tokenizer is not None:                                            │
│   113 │   │   │   # We need to convert a slow tokenizer to build the backend              │
│   114 │   │   │   fast_tokenizer = convert_slow_tokenizer(slow_tokenizer)                 │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
Exception: stream did not contain valid UTF-8

In [132]:
tokenizer = GPTNeoXTokenizerFast(
    vocab_file="/fsx/jp-llm/stablelm-jp/novelai-spm-vocab.json",
    merges_file="/fsx/jp-llm/stablelm-jp/novelai-spm-merges.txt",
    tokenizer_file="/fsx/jp-llm/stablelm-jp/novelai.model",
    errors="ignore",
)

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_2042734/3017839043.py:1 in <module>                                        │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2042734/3017839043.py'               │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/transformers/models/gpt_neox/ │
│ tokenization_gpt_neox_fast.py:113 in __init__                                             │
│                                                                                           │
│   110 │   │   add_prefix_space=False,                                                     │
│   111 │   │   **kwargs,                                                                   │
│   112 │   ):                                                                              │
│ ❱ 113 │   │   super().__init__(                                                           │
│   114 │   │   │   vocab_file,                                                             │
│   115 │   │   │   merges_file,                                                            │
│   116 │   │   │   tokenizer_file=tokenizer_file,                                          │
│                                                                                           │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/transformers/tokenization_uti │
│ ls_fast.py:111 in __init__                                                                │
│                                                                                           │
│   108 │   │   │   fast_tokenizer = copy.deepcopy(tokenizer_object)                        │
│   109 │   │   elif fast_tokenizer_file is not None and not from_slow:                     │
│   110 │   │   │   # We have a serialization from tokenizers which let us directly build t │
│ ❱ 111 │   │   │   fast_tokenizer = TokenizerFast.from_file(fast_tokenizer_file)           │
│   112 │   │   elif slow_tokenizer is not None:                                            │
│   113 │   │   │   # We need to convert a slow tokenizer to build the backend              │
│   114 │   │   │   fast_tokenizer = convert_slow_tokenizer(slow_tokenizer)                 │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
Exception: stream did not contain valid UTF-8

In [14]:
from transformers import AutoModelForCausalLM

In [15]:
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt-1b", use_fast=False, add_eos_token=True)

In [16]:
model = AutoModelForCausalLM.from_pretrained(
    "rinna/japanese-gpt-1b",
    load_in_8bit=True,
    device_map="auto",
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


In [17]:
type(model)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel